In [1]:
import numpy as np

In [2]:
def activation(s):
    return 0 if s <= 0 else s

def derivativeActivation(s):
    # return 0 if s <= 0 else 1
    return (1 - tanh(s) ** 2)

def outputf(s):
    return (1) / ((1) + np.exp(-s))

def derivativeOutput(s):
    return (outputf(s)) * (1 - outputf(s))

def errorf(x_L, y):
    if y == 1:
        return np.log(x_L)
    else:
        return -np.log(1 - x_L)
    
def tanh(s):
    return np.tanh(s)

In [3]:
X = [1, 2]
W = [
    [[0.1,0.2],
     [0.3,0.4]],
    [[0.2],
     [1],
     [-3]],
    [[1],
     [2]]
]

In [4]:
def forwardPropagation(x, weights):
    Xl = np.array(x)
    W = np.array(weights)
    S = []
    X = [x]
    for index,l in enumerate(W):
        wl = np.array(l)
        sl = np.transpose(wl).dot(Xl)
        Xl_before_activation = sl
        if index != len(W) - 1:
            activation_function = np.vectorize(tanh)
            Xl = activation_function(Xl_before_activation)
            Xl = np.insert(Xl, 0, 1, axis=0)
        else:
            output_function = np.vectorize(tanh)
            Xl = output_function(Xl_before_activation)
        X.append(Xl)
        S.append(sl)
    np.delete
    return np.array(X),np.array(S)

In [5]:
x,s = forwardPropagation(X,W)
for i in s:
    print(i)
print('\n',s)

[0.7 1. ]
[-1.48041469]
[-0.80309131]

 [array([0.7, 1. ]) array([-1.48041469]) array([-0.80309131])]


In [83]:
def backPropagation(X, y_n, s, weights):
    delta = 0
    g = []
    W = np.array(weights)
    X = np.array(X)
    for layer, Xl in enumerate(reversed(X)):
        if layer == 0:
            delta = 2 * (Xl[0] - y_n) * derivativeActivation(s[-1][0])
            g.append(np.array(delta))
        elif layer != len(X) - 1:
            Wl = np.array(weights[len(X) - layer - 1])
            a = (g[layer - 1].dot(Wl.T)).T
            g.append(a)
            # print('gl-1',g[layer - 1])
            # print('WlT',Wl.T)
            # print(g[layer].shape, Wl.T.shape,'\n')
    return (g)

In [90]:
g = backPropagation(x, 1, s, W)
print(g[2])

gl-1 -1.8548643739176334
WlT [[1 2]]
(2, 1) (1, 2) 

gl-1 [[-1.85486437]
 [-3.70972875]]
WlT [[ 0.2  1.  -3. ]]
(3, 2) (1, 3) 

[[-0.37097287 -0.74194575]
 [-1.85486437 -3.70972875]
 [ 5.56459312 11.12918624]]


In [35]:
a = np.array([[1],[2],[3]])
i = np.identity(3) * 4
a.T.dot(i)

array([[ 4.,  8., 12.]])

In [72]:
g = backPropagation(x, 1, s, W)

w = np.array(W[1])
print(g[1], g[1].shape)
print(w, w.shape)
g[1] = np.delete(g[1], 0, 0)
g[1].dot(w)

gl-1 -1.8548643739176334
WlT [[1 2]]
(2, 1) (1, 2) 

gl-1 [[-1.85486437]
 [-3.70972875]]
WlT [[ 0.2  1.  -3. ]]
(3, 2) (1, 3) 

gl-1 [[-0.37097287 -0.74194575]
 [-1.85486437 -3.70972875]
 [ 5.56459312 11.12918624]]
WlT [[0.1 0.3]
 [0.2 0.4]]
(2, 3) (2, 2) 

[[-1.85486437]
 [-3.70972875]] (2, 1)
[[ 0.2]
 [ 1. ]
 [-3. ]] (3, 1)


ValueError: shapes (1,1) and (3,1) not aligned: 1 (dim 1) != 3 (dim 0)

In [71]:
g = backPropagation(x, 1, s, W)

w = np.array(W[2])
print(g[2], g[2].shape)
print(w, w.shape)
g[2] = np.delete(g[2], 0, 0)
g[2].dot(w)

gl-1 -1.8548643739176334
WlT [[1 2]]
(2, 1) (1, 2) 

gl-1 [[-1.85486437]
 [-3.70972875]]
WlT [[ 0.2  1.  -3. ]]
(3, 2) (1, 3) 

gl-1 [[-0.37097287 -0.74194575]
 [-1.85486437 -3.70972875]
 [ 5.56459312 11.12918624]]
WlT [[0.1 0.3]
 [0.2 0.4]]
(2, 3) (2, 2) 

[[-0.37097287 -0.74194575]
 [-1.85486437 -3.70972875]
 [ 5.56459312 11.12918624]] (3, 2)
[[1]
 [2]] (2, 1)


array([[-9.27432187],
       [27.82296561]])